In [4]:
import pandas as pd 

temp=pd.read_csv("final_temp_dust")
temp.head()

,etl_ymd_dateType,weekday,max_temp,min_temp,pm10_max,pm10_min,pm2_5_max,pm2_5_min
0,2018-01-01,0,5.5,-0.1,48.0,24.0,NaN,NaN
1,2018-01-02,1,7.6,-0.4,122.0,34.0,NaN,NaN
2,2018-01-03,2,4.6,-1.8,58.0,9.0,NaN,NaN
3,2018-01-04,3,5.4,-2.9,48.0,13.0,NaN,NaN
4,2018-01-05,4,7.9,0.5,57.0,16.0,NaN,NaN


In [7]:
dust=pd.read_excel("./data/dust_inform.xlsx",sheet_name=3,skiprows=[0])
dust.head(10)

,날짜,PM10,PM2.5
0,(월-일-시),(㎍/㎥),(㎍/㎥)
1,01-01-01,55,NaN
2,01-01-02,56,NaN
3,01-01-03,43,NaN
4,01-01-04,51,NaN
5,01-01-05,36,NaN
6,01-01-06,38,NaN
7,01-01-07,40,NaN
8,01-01-08,35,NaN
9,01-01-09,36,NaN


In [8]:
dust.drop([0],axis=0,inplace=True)
dust.head()

,날짜,PM10,PM2.5
1,01-01-01,55,NaN
2,01-01-02,56,NaN
3,01-01-03,43,NaN
4,01-01-04,51,NaN
5,01-01-05,36,NaN


우선 합치기 위해 날짜를 원본 데이터의 형태로 만들어 줍니다. 


In [9]:
from datetime import datetime
dust['etl_ymd_dateType']=dust['날짜'].apply(lambda x: "2018"+"-"+x[0:2]+"-"+x[3:5])
dust['etl_ymd_dateType']=dust['etl_ymd_dateType'].apply(lambda x: datetime.strptime(x,"%Y-%m-%d"))
dust.head()

,날짜,PM10,PM2.5,etl_ymd_dateType
1,01-01-01,55,NaN,2018-01-01
2,01-01-02,56,NaN,2018-01-01
3,01-01-03,43,NaN,2018-01-01
4,01-01-04,51,NaN,2018-01-01
5,01-01-05,36,NaN,2018-01-01


이제 groupby 해준후 최소 최대 값을 집어 넣어줍니다.

In [10]:
pm10_max=dust.groupby("etl_ymd_dateType")["PM10"].max()
pm2_5_max=dust.groupby("etl_ymd_dateType")["PM2.5"].max()
pm10_min=dust.groupby("etl_ymd_dateType")["PM10"].min()
pm2__min=dust.groupby("etl_ymd_dateType")["PM2.5"].min()
date=dust['etl_ymd_dateType'].unique()


In [16]:
df=pd.DataFrame()
df['etl_ymd_dateType']=date
df['zungu_pm10_max']=pm10_max.values
df['zungu_pm10_min']=pm10_min.values
df['zungu_pm2_5_max']=pm2_5_max.values
df['zungu_pm2_5_min']=pm2__min.values
df.head()

,etl_ymd_dateType,zungu_pm10_max,zungu_pm10_min,zungu_pm2_5_max,zungu_pm2_5_min
0,2018-01-01,56.0,25.0,NaN,NaN
1,2018-01-02,122.0,37.0,NaN,NaN
2,2018-01-03,70.0,17.0,NaN,NaN
3,2018-01-04,49.0,20.0,NaN,NaN
4,2018-01-05,67.0,24.0,NaN,NaN


In [12]:
temp.head()

,etl_ymd_dateType,weekday,max_temp,min_temp,pm10_max,pm10_min,pm2_5_max,pm2_5_min
0,2018-01-01,0,5.5,-0.1,48.0,24.0,NaN,NaN
1,2018-01-02,1,7.6,-0.4,122.0,34.0,NaN,NaN
2,2018-01-03,2,4.6,-1.8,58.0,9.0,NaN,NaN
3,2018-01-04,3,5.4,-2.9,48.0,13.0,NaN,NaN
4,2018-01-05,4,7.9,0.5,57.0,16.0,NaN,NaN


In [15]:
print(temp['etl_ymd_dateType'].dtype)
print(df['etl_ymd_dateType'].dtype)

datetime64[ns]
datetime64[ns]


merge해주기 전에 merge의 기준이 되는 etl_ymd를 datetime으로 바꿔줍시다. 

In [14]:
temp['etl_ymd_dateType']=temp['etl_ymd_dateType'].apply(lambda x: datetime.strptime(x,"%Y-%m-%d"))
temp.head()

,etl_ymd_dateType,weekday,max_temp,min_temp,pm10_max,pm10_min,pm2_5_max,pm2_5_min
0,2018-01-01,0,5.5,-0.1,48.0,24.0,NaN,NaN
1,2018-01-02,1,7.6,-0.4,122.0,34.0,NaN,NaN
2,2018-01-03,2,4.6,-1.8,58.0,9.0,NaN,NaN
3,2018-01-04,3,5.4,-2.9,48.0,13.0,NaN,NaN
4,2018-01-05,4,7.9,0.5,57.0,16.0,NaN,NaN


이제 merge를 해줍시다

In [17]:
new_df=pd.merge(temp,df,on="etl_ymd_dateType")
new_df.head()

,etl_ymd_dateType,weekday,max_temp,min_temp,pm10_max,pm10_min,pm2_5_max,pm2_5_min,zungu_pm10_max,zungu_pm10_min,zungu_pm2_5_max,zungu_pm2_5_min
0,2018-01-01,0,5.5,-0.1,48.0,24.0,NaN,NaN,56.0,25.0,NaN,NaN
1,2018-01-02,1,7.6,-0.4,122.0,34.0,NaN,NaN,122.0,37.0,NaN,NaN
2,2018-01-03,2,4.6,-1.8,58.0,9.0,NaN,NaN,70.0,17.0,NaN,NaN
3,2018-01-04,3,5.4,-2.9,48.0,13.0,NaN,NaN,49.0,20.0,NaN,NaN
4,2018-01-05,4,7.9,0.5,57.0,16.0,NaN,NaN,67.0,24.0,NaN,NaN


In [18]:
new_df.to_csv("final_temp_dust",index=False)

In [19]:
df=pd.read_csv("final_temp_dust")
df.head()

,etl_ymd_dateType,weekday,max_temp,min_temp,pm10_max,pm10_min,pm2_5_max,pm2_5_min,zungu_pm10_max,zungu_pm10_min,zungu_pm2_5_max,zungu_pm2_5_min
0,2018-01-01,0,5.5,-0.1,48.0,24.0,NaN,NaN,56.0,25.0,NaN,NaN
1,2018-01-02,1,7.6,-0.4,122.0,34.0,NaN,NaN,122.0,37.0,NaN,NaN
2,2018-01-03,2,4.6,-1.8,58.0,9.0,NaN,NaN,70.0,17.0,NaN,NaN
3,2018-01-04,3,5.4,-2.9,48.0,13.0,NaN,NaN,49.0,20.0,NaN,NaN
4,2018-01-05,4,7.9,0.5,57.0,16.0,NaN,NaN,67.0,24.0,NaN,NaN
